In [0]:
!pip install yfinance

  Using cached yfinance-0.2.38-py2.py3-none-any.whl (72 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached peewee-3.17.5.tar.gz (3.0 MB)
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - error
  ERROR: Command errored out with exit status 1:
   command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-43970eee-6492-4bbd-9ce0-2b67ac59a439/bin/python /local_disk0/.ephemeral_nfs/envs/pythonEnv-43970eee-6492-4bbd-9ce0-2b67ac59a439/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpv78ikabg
       cwd: /tmp/pip-install-vrdfxp1z/peewee_786cdff2ff454f54990b6c92f4b534c8
  Complete output (38 lines):
  
  Error compiling Cython file:
  ------------------------------------------------------------
  ...
  
  
  # We define an xConnect function, but leave xCreate NULL so that the
  # table-function can be c

In [0]:
# Listar os diretorios
dbutils.fs.ls("/")

Out[5]: [FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/', name='tmp/', size=0, modificationTime=0)]

In [0]:
# Criar novo diretorio
dbutils.fs.mkdirs("/mnt/dados")

Out[7]: True

In [0]:
# Listar os diretorios
dbutils.fs.ls("/mnt")

Out[9]: [FileInfo(path='dbfs:/mnt/dados/', name='dados/', size=0, modificationTime=0)]

In [0]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
import datetime as dtm
from dateutil import parser
from dateutil.relativedelta import relativedelta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit
from pyspark.sql.types import *
import json
import os
from pyspark.sql import functions as f
from pyspark.sql import window as w
from functools import reduce
import functools
from pyspark.sql import DataFrame

In [0]:
# Seleciona Stocks
ticker_list=['MSFT', 'NURO']

# Seleciona Periodo de Coleta
start_date = dtm.datetime(2024,5,9) 
today = date.today()

In [0]:
# Coleta Dados
lista=[]

for tik in ticker_list:
  data = spark.createDataFrame(pdr.get_data_yahoo(tik, start=start_date, end=today))
  data = data.withColumnRenamed('Adj Close', 'Adj_Close').withColumn('StokeName', lit(tik))
  
  lista.append(data)
  #data = yf.Ticker(tik) 
  # Adiciona Novas Colunas
  #data = data.withColumn('StokeName', lit(tik))
  
  #print(data.history(start=start_date, end=today))
data = functools.reduce(DataFrame.union, lista)
display(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Open,High,Low,Close,Adj_Close,Volume,StokeName
410.57000732421875,412.7200012207031,409.1000061035156,412.32000732421875,412.32000732421875,14670400,MSFT
4.539999961853027,4.550000190734863,4.409999847412109,4.409999847412109,4.409999847412109,3400,NURO


In [0]:

# Salva Dados no Formato Tabela
data.write.mode('overwrite').saveAsTable('Stock')

In [0]:

# Salva Dados no Formato Parquet
data.write.format('parquet').mode('overwrite').save('/mnt/dados/Stock.parquet')

In [0]:
# Lista Diretório
dbutils.fs.ls('/mnt/dados')

Out[19]: [FileInfo(path='dbfs:/mnt/dados/Stock.parquet/', name='Stock.parquet/', size=0, modificationTime=0)]

In [0]:
# Lendo Arquivo Parquet
df_stock = spark.read.format('parquet').load('/mnt/dados/Stock.parquet')
display(df_stock)

Open,High,Low,Close,Adj_Close,Volume,StokeName
410.57000732421875,412.7200012207031,409.1000061035156,412.32000732421875,412.32000732421875,14670400,MSFT
4.539999961853027,4.550000190734863,4.409999847412109,4.409999847412109,4.409999847412109,3400,NURO


In [0]:
df_stock.show()

+------------------+-----------------+-----------------+------------------+------------------+--------+---------+
|              Open|             High|              Low|             Close|         Adj_Close|  Volume|StokeName|
+------------------+-----------------+-----------------+------------------+------------------+--------+---------+
|410.57000732421875|412.7200012207031|409.1000061035156|412.32000732421875|412.32000732421875|14670400|     MSFT|
| 4.539999961853027|4.550000190734863|4.409999847412109| 4.409999847412109| 4.409999847412109|    3400|     NURO|
+------------------+-----------------+-----------------+------------------+------------------+--------+---------+



In [0]:
%sql

select * from Stock;

Open,High,Low,Close,Adj_Close,Volume,StokeName
410.57000732421875,412.7200012207031,409.1000061035156,412.32000732421875,412.32000732421875,14670400,MSFT
4.539999961853027,4.550000190734863,4.409999847412109,4.409999847412109,4.409999847412109,3400,NURO
